<a href="https://colab.research.google.com/github/vineet-codes/defi-exploration/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
def get_usernames(r=2):
  inputstring = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0123456789'
  # print(len(inputstring))
  username_list =list(set([inputstring[x:y] for x, y in itertools.combinations(range(len(inputstring) + 1), r = r)]))
  return username_list

len(get_usernames())

2015

Questions: 

1. How does unstake work [user has to claim it ?]: Not implemented at this point of time.

## Four main classes

1. Vault
2. Stakes
3. Rewards
4. Users

In [3]:
class Vault:

  def __init__(self, name):
    # pool is of the form {user: [(amount_1, tick_1), (amount_2, tick_2)]}
    self.pool = {}

    self.name = name
    # self.rewards_pool = rewards_pool
    # tick is used for having a contcept of time in our analysis
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def balanceOf(self, user):
    user_txns = self.pool.get(user,[])
    if(len(user_txns) == 0):
      return 0 #f'It seems {user} has made no txns in to the Vault'
    return sum([amount for (amount, _) in user_txns])
    

  def shareOfUser(self, user):
    """
    return: users shares of the total pool
    """
    return self.balanceOf(user) / self.tvl()

  def deposit(self, amount, user):
    """
    return: tuple (user, updated balance)
    """
    if (user in self.pool.keys()):
      user_txns = self.pool.get(user, []) 
      user_txns.append((amount, self.tick))
      self.pool[user] = user_txns
    else:
      self.pool[user] = [(amount, self.tick)]
    return user, self.pool[user]

  def withdraw(self, withdraw_amount, user):
    """
    return: tuple (user, updated balance)
    """
    if(user not in self.pool.keys()):
      return f'{user} has not deposited in the vault'
    
    if(self.balanceOf(user) < withdraw_amount):
      return f'Not enough balance, {user} can one withdraw upto: {self.balanceOf(user)}'
    
    # db update
    # self.pool[user] = self.pool[user] - withdraw_amount
    self.pool[user].append((-1* withdraw_amount, self.tick))

    return user,self.balanceOf(user)

  def tvl(self):
    result = 0
    for user, txns in self.pool.items():
      user_balance = self.balanceOf(user)
      result += user_balance
    return result
      
    return sum([v for _,v in self.pool.items()])
  
  def get_pool(self):
    return self.pool



class Stakes:

  def __init__(self, name="YOP_LOCKBOX"):

    # pool is of the form {user: [(amount_1, duration_1, tick_1), (amount_2, duration_2, tick_2), (amount_1, duration_1, tick_2)]}
    self.pool = {}

    self.name = name
    # self.rewards_pool = rewards_pool
    # tick helps with time travel
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def stake(self, user, amount, duration):
    """
    duratrion: in months
    """
    # assert self.tick == tick

    user_stakes = self.pool.get(user, [])
    # print(user_stakes)
    user_stakes.append((amount, duration, self.tick))
    self.pool[user] = user_stakes
    # self.rewards_pool.tick()
    return user, self.pool.get(user, [])

  def unstake(self):
    pass


  def get_pool(self):
    return self.pool

  def pool_size(self):
    return sum([amount*duration for user in self.pool.keys() for amount, duration, _ in self.pool[user]])

  def shareOfUser(self, user):
    totalPoolSize = self.pool_size()
    if(totalPoolSize == 0):
      return 0
    userPoolSize = sum([amount*duration for amount, duration, _ in self.pool.get(user, [(0,0,0)])])
    return userPoolSize / totalPoolSize



class Rewards:

  def __init__(self, vault, stakes, 
               weights=[0.2,0.8], 
               vault_stake_reward_split = [0.75, 0.25], 
               yearly_emissions = 3891931,
               yop_token_price = 1):
    
    self.tick = 0
    
    self.yearly_emissions = yearly_emissions

    self.vault = vault
    self.stakes = stakes

    self.weights = weights
    self.vault_stake_reward_split = vault_stake_reward_split
    self.yop_token_price = yop_token_price

    # list to store history state per tick
    self.reward_per_tick_history = []

  
  def update_tick(self, step):
    self.tick += step


  def calculate_rewards(self):

    result = {}
    
    total_power = 0
    for user in list(set(self.vault.pool.keys())):
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      total_power += vault_boost_multiplier * self.vault.balanceOf(user)

    # print(total_power)

    for user in list(set(self.vault.pool.keys())):
      # print("=========")
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      # print(f'{user} vault boost multiplier is: {vault_boost_multiplier}')
      # x = (vault_boost_multiplier * self.vault.balanceOf(user) / total_power) * ((self.vault_stake_reward_split[0]/4)*self.yearly_emissions) * (self.yop_token_price)
      x = ((vault_boost_multiplier * self.vault.balanceOf(user)) / total_power) * ((0.2)*self.yearly_emissions) * (self.yop_token_price)
      # print(f"{user} is getting {(x / self.vault.balanceOf(user))*100}% APY in the vault")
      result[user] = (x / self.vault.balanceOf(user))*100
    return result
 
    
  
  def tick_fn(self, step):


    # update clocks for everyone
    self.update_tick(step)
    self.vault.update_tick(step)
    self.stakes.update_tick(step)

    # TODO: calculate the rewards earned in this tick

    # TODO: unstake and calculate stake rewards to be persisted

    # calculate the current vault reward state to be persisted
    state = {}

    # print(f"===Tick: {self.tick}====")
    # print(self.calculate_rewards())
    # print("Vault pool",self.vault.pool)
    # print("Stake Pool", self.stakes.pool)

    state[f'apys_{self.tick}'] = self.calculate_rewards()
    state['vault_pool'] = self.vault.pool.copy()
    state['stake_pool'] = self.stakes.pool.copy()

    self.reward_per_tick_history.append(state)
    # print("========")

    
    
      

  


# v = Vault()
# print(v.tvl())
# print(v.balanceOf('dfg'))
# v.deposit(5000, 'e')
# print(v.tvl())

class User:

  def __init__(self, name):
    self.name = name

  def __repr__(self):
    return f'{self.name}'

  def vault_rewards(self):
    pass

  def stake_rewards(self):
    pass

In [4]:
usdc_vault = Vault('test')
yop_stakes = Stakes()
vineet = User("Vineet")
john = User("John")
anar = User("Anar")

reward_calculator = Rewards(usdc_vault, yop_stakes)

reward_calculator.tick_fn(1)

# users make deposit into vault
usdc_vault.deposit(50_000, vineet)
# yop_stakes.stake(john, 5000, 12)


reward_calculator.tick_fn(1)

usdc_vault.deposit(50_000, vineet)

reward_calculator.tick_fn(1)

usdc_vault.deposit(60_000, john)

# users stake some yop
yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)

yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)


reward_calculator.tick_fn(1)

In [5]:
usdc_vault.pool, usdc_vault.balanceOf(anar)

({John: [(60000, 3)], Vineet: [(50000, 1), (50000, 2)]}, 0)

In [6]:
usdc_vault.shareOfUser(vineet) + usdc_vault.shareOfUser(john) == 1

True

In [7]:
usdc_vault.withdraw(100000, vineet), usdc_vault.balanceOf(vineet), usdc_vault.tvl(), yop_stakes.get_pool()

((Vineet, 0), 0, 60000, {Vineet: [(5000, 12, 3), (5000, 12, 4)]})

##Toy Sheet setup: Replication our excel sheet

In [8]:
def toy_sheet_setup(steps = 100):

  # create vault and stake pool
  usdc_vault = Vault('test vault')
  yop_stakes = Stakes()
  rewards = Rewards(usdc_vault, yop_stakes)

  # create users
  vineet, john, anar = User("Vineet"), User("John"), User("Anar")

  # make some txns in vaults and stakes in some ticks
  # tick
  rewards.tick_fn(1)

  # users make deposit into vault
  usdc_vault.deposit(670_000, vineet)
  yop_stakes.stake(john, 100_000, 5)

  #tick 
  rewards.tick_fn(1)
  
  usdc_vault.deposit(2_000_000, john)

  # users stake some yop
  # yop_stakes.stake(vineet, 5000, 12)
  rewards.tick_fn(1)

  usdc_vault.deposit(5000, anar)

  yop_stakes.stake(anar, 50000, 5)
  rewards.tick_fn(1)

  # yop_stakes.stake(anar, 5000, 12)
  rewards.tick_fn(1)



  return rewards, usdc_vault, yop_stakes, [vineet, john, anar]

In [9]:
rewards, usdc_vault, yop_stakes, [vineet, john, anar] = toy_sheet_setup()

In [10]:
usdc_vault.pool, yop_stakes.pool, rewards.tick, rewards.calculate_rewards()

({Anar: [(5000, 3)], John: [(2000000, 2)], Vineet: [(670000, 1)]},
 {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
 5,
 {Anar: 14.840511248388019,
  John: 37.94960412560259,
  Vineet: 2.7838943172431097})

In [11]:
rewards.reward_per_tick_history

[{'apys_1': {}, 'stake_pool': {}, 'vault_pool': {}},
 {'apys_2': {Vineet: 116.1770447761194},
  'stake_pool': {John: [(100000, 5, 1)]},
  'vault_pool': {Vineet: [(670000, 1)]}},
 {'apys_3': {John: 38.24237769141653, Vineet: 2.0206934584581293},
  'stake_pool': {John: [(100000, 5, 1)]},
  'vault_pool': {John: [(2000000, 2)], Vineet: [(670000, 1)]}},
 {'apys_4': {Anar: 14.840511248388019,
   John: 37.94960412560259,
   Vineet: 2.7838943172431097},
  'stake_pool': {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
  'vault_pool': {Anar: [(5000, 3)],
   John: [(2000000, 2)],
   Vineet: [(670000, 1)]}},
 {'apys_5': {Anar: 14.840511248388019,
   John: 37.94960412560259,
   Vineet: 2.7838943172431097},
  'stake_pool': {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
  'vault_pool': {Anar: [(5000, 3)],
   John: [(2000000, 2)],
   Vineet: [(670000, 1)]}}]

In [12]:
common_users = list(set(rewards.vault.pool.keys()).intersection(set(rewards.stakes.pool.keys())))

In [13]:
common_users

[Anar, John]

## WIP: Setup Optimization simulation

In [14]:
import random

`setup_sim` sets up the simulation where for each `step` a tick is performed and then a randomly chosen user performs a randomly chosen action out of `['deposit', 'withdraw', 'stake', 'unstake']`. Then all the data is return ofr analysis.

In [21]:
def setup_sim(steps=300):

  actions = ['deposit', 'withdraw', 'stake', 'unstake']

  # instantiate vaults and staking pool
  usdc_vault = Vault('USDC_GENESIS')
  yop_stakes = Stakes()
  rewards = Rewards(usdc_vault, yop_stakes)

  action_history = []

  # generate random users from population of all users
  usernames = get_usernames()
  user_population = [User(username) for username in usernames]

  for tick in range(1, steps+1):
    # update the tick
    rewards.tick_fn(1)
    # for each tick
    #  pick a random user and a random action he takes
    user = random.choice(user_population[:20])
    action_name = random.choice(actions)

    # print(f"{user} {action_name}'s at tick {rewards.tick}")
    action_history.append({user: action_name, 'tick': rewards.tick})

    # execute the action
    if(action_name == 'deposit'):
      # deposit: user deposits a random amount in vault
      usdc_vault.deposit(10000, user)
    elif (action_name == 'withdraw'):
      balance = usdc_vault.balanceOf(user)
      usdc_vault.withdraw(balance/8, user)
    elif (action_name == 'stake'):
      yop_stakes.stake(user, 1000, 12)
    else:
      # unstake
      pass
  # print(rewards.calculate_rewards())
  return rewards, user_population, action_history

In [22]:
rewards, user_population, action_history = setup_sim()

In [23]:
len(rewards.vault.pool.keys()), rewards.vault.tvl()

(20, 605884.3994140625)

In [24]:
rewards.tick, rewards.vault.tick, rewards.stakes.tick

(300, 300, 300)

In [25]:
len(action_history)

300

What actions by which users were taken in this run of the simulation ? 

We record this in `action_history` array. Need to figure out a way of plotting this random walk. It is a random walk because each run of the simulation will generate a random actions by randomly picked users. The idea is to average over all the runs to idetify key variables.

In [26]:
action_history[:10]

[{lmnopqrstuvw: 'withdraw', 'tick': 1},
 {qrstuvwx: 'withdraw', 'tick': 2},
 {XYZabscdefghijklmnopqrstuvwxyz01234567: 'withdraw', 'tick': 3},
 {qrstuvwx: 'unstake', 'tick': 4},
 {FGHIJKLMNO: 'withdraw', 'tick': 5},
 {ABCDEFGHIJKLMNOPQRSTUVWXYZabs: 'withdraw', 'tick': 6},
 {UVWXYZabscdefghijk: 'deposit', 'tick': 7},
 {NOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz012: 'withdraw', 'tick': 8},
 {FGHIJKLMNO: 'withdraw', 'tick': 9},
 {defghijkl: 'deposit', 'tick': 10}]

What is historical apy's of users over the run of the simulation ? 

We have `rewards.reward_per_tick_history` array which has all this data. We need to plot this in an easy consumable way.

In [32]:
rewards.reward_per_tick_history[-1][f'apys_{300}'], rewards.reward_per_tick_history[-2][f'apys_{299}']

({ABCDEFGHIJKLMNOP: 118.76574179889894,
  ABCDEFGHIJKLMNOPQRSTUVWXYZabs: 236.1781324092707,
  BCDEFGHIJKLMNOPQRSTUVWXYZab: 153.38809414918853,
  FGHIJKLMNO: 172.5054239410632,
  FGHIJKLMNOPQRSTUVWXYZ: 112.26507952779863,
  JKLMNOPQRSTUVWXYZab: 127.10028797608204,
  KLMNOPQRSTUVWXYZabs: 168.27953042403553,
  MNOPQRSTUVWXY: 78.0273841989353,
  NOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz012: 185.04086414257287,
  QRSTUVWXYZabscdefghijklmnopq: 128.72545354385713,
  STUVWXYZabscd: 92.83149891824338,
  UVWXYZabscdefghijk: 140.88880209017373,
  WXYZabsc: 75.10871950578824,
  XYZabscdefghijklmnopqrstuvwxyz01234567: 104.39486636997988,
  defghijkl: 120.23479261833833,
  jklmnopqrst: 82.26299618882882,
  lmnopqrstuvw: 139.01890464648335,
  qrstuvwx: 102.71308598659006,
  stuvwxyz012: 88.91576198262207,
  tuvwxyz012: 153.65990516331416},
 {ABCDEFGHIJKLMNOP: 121.00958107373891,
  ABCDEFGHIJKLMNOPQRSTUVWXYZabs: 239.71576602467155,
  BCDEFGHIJKLMNOPQRSTUVWXYZab: 155.77457162299513,
  FGHIJKLMNO: 175.92